# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
data = pd.read_csv("assignment3_creditcard.csv")

In [4]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score


X = data.iloc[:,:-1]
y = data.Class

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 생성 및 훈련
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = logreg_model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# 혼동 행렬(confusion matrix) 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 99.88%
Confusion Matrix:
[[5688    0]
 [   7   41]]


Accuracy 값은 굉장히 높게 도출됨.

In [11]:
data.Class.value_counts()

0    28432
1      246
Name: Class, dtype: int64

그러나 class 불균형이 심하기 때문에 accuracy로는 모델의 성능을 제대로 측정할 수 없음.

보다 나은 성능지표를 찾기 위해 classification_report 함수를 활용

In [14]:
# 분류 보고서 출력 (정밀도, 재현율, F1-score)
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5688
           1       1.00      0.85      0.92        48

    accuracy                           1.00      5736
   macro avg       1.00      0.93      0.96      5736
weighted avg       1.00      1.00      1.00      5736



classification report에 명시된 평가지표를 통해 모델의 성능을 평가한 결과, 클래스 0에 대해서는 매우 높은 정확도를 보이고 있지만, 클래스 1에 대해서는 recall 값이 낮게 나타난다는 사실을 확인할 수 있음.

이는 클래스 1이 클래스 0에 비해 더 적게 분포하고 있어서 발생하는 현상이기 때문에 실제 클래스 1인 경우를 더 잘 식별할 수 있도록 모델을 조정해야 함. (클래스 1에 대한 f1-score 값을 높여야 함)

In [41]:
# 테스트 데이터에 대한 예측 확률
y_prob = logreg_model.predict_proba(X_test)[:, 1]

# 최적의 threshold 값 도출을 위한 반복 예측 작업 수행
threshold_list = []

# 0.1부터 0.9 사이 500개의 threshold 후보군을 대상으로 반복 수행
for threshold in np.linspace(0.1, 0.9, 500):
    y_pred_custom = (y_prob >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_custom)
    threshold_list.append((f1, threshold))

threshold_list.sort(reverse=True)

print('f1-score 기준 상위 5개 threshold 값')

for i in range(5):
    print(f'{i+1}위: threshold = {threshold_list[i][1]} / f1_score = {threshold_list[i][0]}')

f1-score 기준 상위 5개 threshold 값
1위: threshold = 0.2779559118236473 / f1_score = 0.9565217391304348
2위: threshold = 0.2763527054108217 / f1_score = 0.9565217391304348
3위: threshold = 0.274749498997996 / f1_score = 0.9565217391304348
4위: threshold = 0.2731462925851703 / f1_score = 0.9565217391304348
5위: threshold = 0.2715430861723447 / f1_score = 0.9565217391304348


threshold가 0.278 가량일 때 최적의 성능 (f1_score = 0.957)이 도출됨을 알 수 있음.

In [42]:
threshold = threshold_list[0][1]

y_pred_custom = (y_prob >= threshold).astype(int)

class_report = classification_report(y_test, y_pred_custom)
print('Classification Report:')
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5688
           1       1.00      0.92      0.96        48

    accuracy                           1.00      5736
   macro avg       1.00      0.96      0.98      5736
weighted avg       1.00      1.00      1.00      5736



precision 값은 유지시키면서 recall 값을 7% 가량 상승시켰음 => 확연한 성능 개선